In [2]:
import pandas as pd
import torch as th

import json

## Datasets

In [7]:
# Training
rs = 42

with open('data/alpaca_data_cleaned.json', 'r') as f:
    alpaca = json.load(f)

alpaca = pd.DataFrame(alpaca)
alpaca = alpaca.dropna()[alpaca['input'] == '']
alpaca = alpaca.sample(12500, random_state=rs)

In [18]:
alpaca_train = alpaca.iloc[:10000].drop('input', axis=1)
alpaca_eval = alpaca.iloc[10000:12000].drop('input', axis=1)
alpaca_heldout = alpaca.iloc[12000:].drop('input', axis=1)

In [20]:
alpaca_train.to_json(orient='records', path_or_buf='training/alpaca_train_base.json')
alpaca_eval.to_json(orient='records', path_or_buf='evaluation/alpaca_eval.json')
alpaca_heldout.to_json(orient='records', path_or_buf='evaluation/alpaca_heldout.json')

In [21]:
with open('data/safe_instructions.json', 'r') as f:
    safe_inst = json.load(f)

safe_inst = pd.DataFrame(safe_inst)

In [22]:
safe_ins_1 = safe_inst.iloc[:100]
safe_ins_3 = safe_inst.iloc[:300]
safe_ins_5 = safe_inst.iloc[:500]
safe_ins_10 = safe_inst.iloc[:1000]
safe_ins_20 = safe_inst.iloc[:2000]

In [23]:
pd.concat([alpaca_train, safe_ins_1], axis=0).sample(frac=1, random_state=rs).drop('input', axis=1).to_json(orient='records', path_or_buf='training/alpaca_train_100.json')
pd.concat([alpaca_train, safe_ins_3], axis=0).sample(frac=1, random_state=rs).drop('input', axis=1).to_json(orient='records', path_or_buf='training/alpaca_train_300.json')
pd.concat([alpaca_train, safe_ins_5], axis=0).sample(frac=1, random_state=rs).drop('input', axis=1).to_json(orient='records', path_or_buf='training/alpaca_train_500.json')
pd.concat([alpaca_train, safe_ins_10], axis=0).sample(frac=1, random_state=rs).drop('input', axis=1).to_json(orient='records', path_or_buf='training/alpaca_train_1000.json')
pd.concat([alpaca_train, safe_ins_20], axis=0).sample(frac=1, random_state=rs).drop('input', axis=1).to_json(orient='records', path_or_buf='training/alpaca_train_2000.json')

In [29]:
# Generate confiigs
configs = []

# Add Llamas
for repo, model in zip(["meta-llama", "mistralai"], ['Llama-2-7b-hf', 'Mistral-7B-v0.1']):
    for i, safe in enumerate(['base', '100', '300', '500', '1000', '2000']):
        for j in range(3):
            configs.append({
                "train_data_general_path": f"training/alpaca_train_{safe}.json",
                "output_model_name": f"{model}-lora-{safe}-rs-{j+1}",
                "base_repository": "safety-lora",
                "base_model": f"{repo}/{model}",
                "dev_data_path": "evaluation/alpaca_eval.json"
            })

with open('configs.json', 'w') as f:
    json.dump(configs, f, indent=4)